# Boilerplate

In [1]:
# python internal 
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
print(os.getcwd())
import random
import re
import socket
print(socket.gethostname())
import shutil
import subprocess
import sys
# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta
# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

/mnt/home/pleung/projects/bistable_bundle/r3/hinges
dig95


# Flo's original approach:
7. Two-state design: 
`/home/flop/switch/5thround/DHRs/msd7/msd_scripts/` has the scripts



# I need to add a few things to Flo's method.
I will use the serialization build of PyRosetta to enable recording user defined info about the designs.  
This enables downstream inline filtering and data analysis, as well as clustering by lineage.

TODO sequence recovery of parent as a final metric in 05?  
TODO explore `FavorSequenceProfile` instead of `FavorNativeResidue`

# Make functions for looping and labeling to assist downstream penultimate design step

In [3]:
from pyrosetta.distributed.packed_pose.core import PackedPose

def linkres_msd(pdb_path: str, **kwargs) -> str:
    """Load pose. Design with linkres x3: state_X_nocys, state_Y_nocys,
    state_XY_nocys. Dump all 3 poses

    Args:
        inputs (list): The poses corresponding to each state. The poses need 
        to be the same length, and if they are not the same sequence the 
        sequence of the first pose in the list will be threaded unless 
        poly_val is set to true.
        to_design (list): The list of residues in the first pose allowed to be
        designed. Nearby residues to these will be allowed to repack and all 
        others will be locked, and this logic is extended to the linked 
        residues on the rest of the poses.
        cutoffs (tuple): The core and surface cutoffs for SASA or the number
        of sidechain neighbors to determine core vs surface. Anything in
        between is boundary.
        use_dssp (bool): Whether to use DSSP to select if a residue is
        helix/loop/sheet. Reccomended to use DSSP. 
        use_sc_neighbors (bool): Whether to use the number of sidechain
        neighbors to determine core vs surface.
        poly_val (bool): Whether to thread polyvaline onto the poses prior to
        starting design. Useful for benchmarking.
        flexbb (bool): Whether to design with a flexible backbone.
        name (str): The prefix of the pdbs dumped.

    Returns:
        message (str): Either an error message or a task completion message.
    """
    from copy import deepcopy
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed import cluster
    from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
    from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
    from pyrosetta.rosetta.protocols.task_operations import LinkResidues
#     from pyrosetta.rosetta.core.pack.task import operation
    from pyrosetta.rosetta.core.pack.task.operation import (
        IncludeCurrent,
        InitializeFromCommandline,
        ExtraRotamersGeneric,
    )
    from pyrosetta.rosetta.protocols.task_operations import (
        LimitAromaChi2Operation
    )
    
    
    try:
        assert pdb_path is not None
    except AssertionError:
        print("Must provide a path to pdb")
    scores = cluster.get_scores_dict(pdb_path)["scores"]
    ppose = io.pose_from_file(pdb_path)
    pose = io.to_pose(ppose)
    if kwargs["out_path"] == None:
        out_path = os.getcwd()
    else: 
        out_path = kwargs["out_path"]
        os.makedirs(os.path.join(os.getcwd(), out_path), exist_ok=True)
    if kwargs["out_state"] == None:
        out_state = "XY"
    else:
        try:
            assert kwargs["out_state"] in ["X","Y","XY"]
            out_state = kwargs["out_state"]        
        except AssertionError:
            print("out_state must be either 'X', 'Y', or 'XY'")
    cys_X = scores["disulfide_at_X"].split(",")
    cys_Y = scores["disulfide_at_Y"].split(",")
    unique_cys = set(cys_X + cys_Y)
    # check whether disulfides are orthogonal
    if len(unique_cys) < 3: # if there are less than 3 CYS we really f'd up at some point
        raise RuntimeError("There should be at least 3 unique CYS")
    elif out_state == "XY": # if out state is XY we're designing all anyway
        design_resis = [int(x) for x in unique_cys]
    elif len(unique_cys) == 4: # if there are 4 unique CYS they are orthogonal, just pick opposite
        if out_state == "X":
            design_resis = [int(x) for x in cys_Y]
        elif out_state == "Y":
            design_resis = [int(x) for x in cys_X]
        else:
            raise RuntimeError
    elif len(unique_cys) == 3: # if there are 3 unique CYS they are sharing one CYS between states
        if out_state == "X":
            design_resis = [int(x) for x in set(cys_Y)-set(cys_X)]
        elif out_state == "Y":
            design_resis = [int(x) for x in set(cys_X)-set(cys_Y)]
        else:
            raise RuntimeError
    else: # there shouldn't be more than 4 unique CYS which is the only way this should happen
        raise RuntimeError("There shouldn't be more than 4 unique CYS")
    length = scores["total_length"]
    resis_matrix = [[x, int(x+length)] for x in design_resis]
    target_resis = ",".join([str(x) for row in resis_matrix for x in row]) # flatten matrix
    xml_string = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn_design" weights="{sfxn}" >
                <Set use_hb_env_dep="true" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="17" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-1.0" />
                <Set approximate_buried_unsat_penalty_natural_corrections1="true" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-7" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_ser_to_helix_bb="1"/>                    
            </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
            <Index name="designable" resnums="{target_resis}" />
            <Neighborhood name="packable" selector="designable" distance="6.0" include_focus_in_subset="true" />
            <Not name="not_designable" selector="designable" />
            <Not name="not_packable" selector="packable" />
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <OperateOnResidueSubset name="design" selector="designable"> # no CYS; GLY; PRO; TRP
                <RestrictAbsentCanonicalAASRLT aas="VILA" />
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="pack" selector="not_designable">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="lock" selector="not_packable">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
        </TASKOPERATIONS>
        <MOVERS>
            <SwitchChainOrder name="delete_Y" chain_order="1"/>
            <SwitchChainOrder name="delete_X" chain_order="2"/>
        </MOVERS>
        <FILTERS>
        </FILTERS>
        <SIMPLE_METRICS>
        </SIMPLE_METRICS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """.format(
        sfxn=scores["sfxn_used"],
        target_resis=target_resis, # ADEFHIKLMNQRSTVY
    )
    # link all designable residues
    linkres = LinkResidues()
    for row in resis_matrix:
        linkres.add_group(",".join([str(x) for x in row]))
    # set taskops by extracting them from the xml_string
    xml_obj = XmlObjects.create_from_string(xml_string)
    design_task = xml_obj.get_task_operation("design")
    pack_task = xml_obj.get_task_operation("pack")
    lock_task = xml_obj.get_task_operation("lock")
    # set taskops
    task_factory = pyrosetta.rosetta.core.pack.task.TaskFactory()
#     task_factory.push_back(operation.IncludeCurrent())
#     task_factory.push_back(operation.InitializeFromCommandline())
    task_factory.push_back(IncludeCurrent())
    task_factory.push_back(InitializeFromCommandline())
    arochi = LimitAromaChi2Operation()
    arochi.include_trp(True)
    task_factory.push_back(arochi)
#     ex1_ex2 = operation.ExtraRotamersGeneric()
    ex1_ex2 = ExtraRotamersGeneric()
    ex1_ex2.ex1(True); ex1_ex2.ex2(True)
    task_factory.push_back(ex1_ex2)
    task_factory.push_back(design_task)
    task_factory.push_back(pack_task)
    task_factory.push_back(lock_task)
    task_factory.push_back(linkres)
#     print(task_factory.create_task_and_apply_taskoperations(new_pose)) # TODO
    # set movemap
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(True)
    mm.set_jump(False)
    sfxn_design = xml_obj.get_score_function("sfxn_design")
    fast_design = FastDesign(scorefxn_in=sfxn_design, standard_repeats=1)
    fast_design.cartesian(False)
    fast_design.set_task_factory(task_factory)
    fast_design.set_movemap(mm)
    fast_design.minimize_bond_angles(False)
    fast_design.minimize_bond_lengths(False)
    fast_design.min_type("lbfgs_armijo_nonmonotone")
    fast_design.ramp_down_constraints(False)
    
    name = pose.pdb_info().name()
    pose = pose.clone()
    fast_design.apply(pose)
    if out_state == "XY":
        pass
    elif out_state == "X":
        delete_Y = xml_obj.get_mover("delete_Y")
        delete_Y.apply(pose)
    elif out_state == "Y":
        delete_X = xml_obj.get_mover("delete_X")
        delete_X.apply(pose)
    else: # this shouldn't occur and is already handled as an exception above
        pass
    basename_no_ext = name.split("/")[-1].replace(".pdb", "", 1)
    if out_path[-1] != "/":
        out_path += "/"
    else:
        pass
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_{out_state}.pdb"))
    return "Linkres multistate design run complete, exiting."


def linkres_msd(pdb_path: str, **kwargs) -> PackedPose:
    """Load pose. Design with linkres x3: state_X_nocys, state_Y_nocys,
    state_XY_nocys. Dump all 3 poses

    Args:
        inputs (list): The poses corresponding to each state. The poses need 
        to be the same length, and if they are not the same sequence the 
        sequence of the first pose in the list will be threaded unless 
        poly_val is set to true.
        to_design (list): The list of residues in the first pose allowed to be
        designed. Nearby residues to these will be allowed to repack and all 
        others will be locked, and this logic is extended to the linked 
        residues on the rest of the poses.
        cutoffs (tuple): The core and surface cutoffs for SASA or the number
        of sidechain neighbors to determine core vs surface. Anything in
        between is boundary.
        use_dssp (bool): Whether to use DSSP to select if a residue is
        helix/loop/sheet. Reccomended to use DSSP. 
        use_sc_neighbors (bool): Whether to use the number of sidechain
        neighbors to determine core vs surface.
        poly_val (bool): Whether to thread polyvaline onto the poses prior to
        starting design. Useful for benchmarking.
        flexbb (bool): Whether to design with a flexible backbone.
        name (str): The prefix of the pdbs dumped.

    Returns:
        message (str): Either an error message or a task completion message.
    """
    from copy import deepcopy
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed import cluster
    from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
    from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
    from pyrosetta.rosetta.protocols.task_operations import LinkResidues
    from pyrosetta.rosetta.core.pack.task.operation import (
        IncludeCurrent,
        InitializeFromCommandline,
        ExtraRotamersGeneric,
    )
    from pyrosetta.rosetta.protocols.task_operations import (
        LimitAromaChi2Operation
    )
    
    try:
        assert pdb_path is not None
    except AssertionError:
        print("Must provide a path to pdb")
    scores = cluster.get_scores_dict(pdb_path)["scores"]
    ppose = io.pose_from_file(pdb_path)
    pose = io.to_pose(ppose)
    if kwargs["out_path"] == None:
        out_path = os.getcwd()
    else: 
        out_path = kwargs["out_path"]
        os.makedirs(os.path.join(os.getcwd(), out_path), exist_ok=True)
    cys_X = scores["disulfide_at_X"].split(",")
    cys_Y = scores["disulfide_at_Y"].split(",")
    unique_cys = set(cys_X + cys_Y)
    # check whether disulfides are orthogonal
    design_resis = [int(x) for x in unique_cys]
    if len(unique_cys) < 3 or len(unique_cys) > 4: # if there are less than 3 CYS or more than 4 we really f'd up at some point
        raise RuntimeError("There should be at least 3 unique CYS, but no more than 4")
    else:
        cys1_X, cys2_X = int(cys_X[0]), int(cys_X[1])
        cys1_Y, cys2_Y = int(cys_Y[0]), int(cys_Y[1])
    length = scores["total_length"]
    resis_matrix = [[x, int(x+length)] for x in design_resis]
    target_resis = ",".join([str(x) for row in resis_matrix for x in row]) # flatten matrix
    xml_string = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn_design" weights="{sfxn}" >
                <Set use_hb_env_dep="true" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="17" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-1.0" />
                <Set approximate_buried_unsat_penalty_natural_corrections1="true" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-7" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_ser_to_helix_bb="1"/>                    
            </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
            <Index name="designable" resnums="{target_resis}" />
            <Neighborhood name="packable" selector="designable" distance="6.0" include_focus_in_subset="true" />
            <Not name="not_designable" selector="designable" />
            <Not name="not_packable" selector="packable" />
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <OperateOnResidueSubset name="design" selector="designable"> # no CYS; GLY; PRO; TRP
                <RestrictAbsentCanonicalAASRLT aas="VILA" />
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="pack" selector="not_designable">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="lock" selector="not_packable">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
        </TASKOPERATIONS>
        <MOVERS>
            <SwitchChainOrder name="delete_Y" chain_order="1"/>
            <SwitchChainOrder name="delete_X" chain_order="2"/>
            <MutateResidue name="cys1_X" target="{cys1_X}" new_res="CYS" />
            <MutateResidue name="cys2_X" target="{cys2_X}" new_res="CYS" />
            <MutateResidue name="cys1_Y" target="{cys1_Y}" new_res="CYS" />
            <MutateResidue name="cys2_Y" target="{cys2_Y}" new_res="CYS" />
            <ForceDisulfides name="restore_X" 
                scorefxn="sfxn_design"
                disulfides="{cys1_X}:{cys2_X}"
                remove_existing="false"
                repack="true" />
            <ForceDisulfides name="restore_Y" 
                scorefxn="sfxn_design"
                disulfides="{cys1_Y}:{cys2_Y}"
                remove_existing="false"
                repack="true" />
            <FastRelax name="relax" 
                scorefxn="sfxn_design"
                repeats="2" 
                relaxscript="MonomerRelax2019"
                />
        </MOVERS>
        <FILTERS>
        </FILTERS>
        <SIMPLE_METRICS>
        </SIMPLE_METRICS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """.format(
        sfxn=scores["sfxn_used"],
        target_resis=target_resis, # ADEFHIKLMNQRSTVY
        cys1_X=cys1_X,
        cys2_X=cys2_X,
        cys1_Y=cys1_Y,
        cys2_Y=cys2_Y,
    )
    # link all designable residues
    linkres = LinkResidues()
    for row in resis_matrix:
        linkres.add_group(",".join([str(x) for x in row]))
    # set taskops by extracting them from the xml_string
    xml_obj = XmlObjects.create_from_string(xml_string)
    design_task = xml_obj.get_task_operation("design")
    pack_task = xml_obj.get_task_operation("pack")
    lock_task = xml_obj.get_task_operation("lock")
    # set taskops
    task_factory = pyrosetta.rosetta.core.pack.task.TaskFactory()
    task_factory.push_back(IncludeCurrent())
    task_factory.push_back(InitializeFromCommandline())
    arochi = LimitAromaChi2Operation()
    arochi.include_trp(True)
    task_factory.push_back(arochi)
    ex1_ex2 = ExtraRotamersGeneric()
    ex1_ex2.ex1(True); ex1_ex2.ex2(True)
    task_factory.push_back(ex1_ex2)
    task_factory.push_back(design_task)
    task_factory.push_back(pack_task)
    task_factory.push_back(lock_task)
    task_factory.push_back(linkres)
    # set movemap
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(True)
    mm.set_jump(False)
    sfxn_design = xml_obj.get_score_function("sfxn_design")
    fast_design = FastDesign(scorefxn_in=sfxn_design, standard_repeats=1)
    fast_design.cartesian(False)
    fast_design.set_task_factory(task_factory)
    fast_design.set_movemap(mm)
    fast_design.minimize_bond_angles(False)
    fast_design.minimize_bond_lengths(False)
    fast_design.min_type("lbfgs_armijo_nonmonotone")
    fast_design.ramp_down_constraints(False)
    
    name = pose.pdb_info().name()
        
    basename_no_ext = name.split("/")[-1].replace(".pdb", "", 1)
    if out_path[-1] != "/":
        out_path += "/"
    else:
        pass
    pose = pose.clone()
    fast_design.apply(pose)
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_XY.pdb")) 
    
    ref_pose = pose.clone()
    # get state X
    delete_Y = xml_obj.get_mover("delete_Y")
    delete_Y.apply(pose)
    ref_X = pose.clone()
    cys1 = xml_obj.get_mover("cys1_X")
    cys2 = xml_obj.get_mover("cys2_X")
    restore = xml_obj.get_mover("restore_X")
    relax = xml_obj.get_mover("relax")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_X.pdb")) 
    # get rmsd
    relax_X = ref_X.clone()
    relax.apply(relax_X)
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_X)
    rmsd = rmsd_calc.calculate(relax_X)
    scores["rmsd_final_X"] = rmsd
    # get state Y
    pose = ref_pose.clone()
    delete_X = xml_obj.get_mover("delete_X")
    delete_X.apply(pose)
    ref_Y = pose.clone()
    cys1 = xml_obj.get_mover("cys1_Y")
    cys2 = xml_obj.get_mover("cys2_Y")
    restore = xml_obj.get_mover("restore_Y")
    relax = xml_obj.get_mover("relax")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_Y.pdb")) 
    # get rmsd
    relax_Y = ref_Y.clone()
    relax.apply(relax_Y)
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_Y)
    rmsd = rmsd_calc.calculate(relax_Y)
    scores["rmsd_final_Y"] = rmsd
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(ref_pose, key, value)
    ppose = io.to_packed(ref_pose)
    return ppose

flags = """
-multithreading:total_threads 4
-quasisymmetry 1
-corrections::beta_nov16 true
-detect_disulf false
-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc
-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5
"""
import pyrosetta
pyrosetta.init(" ".join(flags.replace("\n\t", " ").split()))
linkres_msd("05_selected/tj/good_trp/2021.03.08.22.24.53.141698_5f14d50d11534d65bd713dca7bf0226d.pdb", out_path="test")


PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r274 2020.50+release.1295438 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -multithreading:total_threads 4 -quasisymmetry 1 -corrections::beta_nov16 true -detect_disulf false -holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5 -database /home/pleung/.conda/envs/cereal

# Setup Dask
Trying a adaptive SLURMCluster. to see the dashboard, forward port `8787` to `8000`:  
`local$ ssh -L 8000:localhost:8787 $USER@$HOSTNAME`  
now, the web UI is visible at `localhost:8000`  
if you"re using a local cluster make sure the node this notebook is on has the same 
number of workers as cores

TODO, force stdout to go into logs
TODO `extra=["--lifetime", "170m", "--lifetime-stagger", "4m"]`

In [ ]:
!echo $HOSTNAME
!echo $USER

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    cores=1,
    processes=1,
    job_cpu=1,
    local_directory="$TMPDIR/dask",
    log_directory="/mnt/home/pleung/logs/slurm_logs",
    memory="4GB",
    queue="medium",
    walltime="23:58:00",
    death_timeout=600,
)
print(cluster.job_script())
# scale between 0 and 1000 workers as needed
cluster.adapt(minimum=0, maximum=1020, wait_count=999) 
client = Client(cluster)
client

In [ ]:
# client.close(); cluster.close()

# Set command line options, make tasks and submit to client
Try `nstruct` of 50 to start

In [ ]:
import logging
import pyrosetta.distributed.io as io
from pyrosetta.distributed.cluster.core import PyRosettaCluster
logging.basicConfig(level=logging.INFO)
import random

pickles = glob(os.path.join(os.getcwd(), "04_pickles/DHR*/*.pickle"))
more_pickles = glob(os.path.join(os.getcwd(), "04_pickles/hDHR*/*.pickle"))
pickles.extend(more_pickles)

options = { 
    "-out:level": "300",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    "-dunbrack_prob_buried": "0.8",
    "-dunbrack_prob_nonburied": "0.8", 
    "-dunbrack_prob_buried_semi": "0.8", 
    "-dunbrack_prob_nonburied_semi": "0.8",
}

def create_tasks(pickles, options):
    for pickle_path in pickles:
        tasks = {"options": "-corrections::beta_nov16 true"}
        tasks["extra_options"] = options
        tasks["pickle_path"] = pickle_path
        tasks["np_pen"] = 2
        tasks["set_logging_handler"] = "interactive"
        yield tasks
        
if not os.getenv("DEBUG"):
    output_path = os.path.join(os.getcwd(), "05_msd_score_n50_1trp")
    PyRosettaCluster(
        tasks=create_tasks(pickles, options),
        client=client,
        scratch_dir=output_path,
        output_path=output_path,
        nstruct=50,
        ignore_errors=True,
    ).distribute(protocols=[msd, score])